# Мастерская. Проект.

## Заказчик: *GoProtect*

### Задача: сервис  “Мой Чемпион” помогает спортивным школам фигурного катания, тренерам мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.

#### Этап 1. Исследование данных

In [13]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sentence_transformers.util import semantic_search
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.trainer import SentenceTransformerTrainer
from torch.utils.data import DataLoader
import nlpaug.augmenter.char as nac

In [2]:
df_rough = pd.read_csv('C:/Users/Home/Downloads/data_02_match_school/Примерное_написание.csv')
df_clean = pd.read_csv('C:/Users/Home/Downloads/data_02_match_school/Школы.csv')

In [3]:
# сбор данных о датафрейме:
def data_info(data):
    print(f'''
    ----------------------------------------
    Первые строки датафрейма:
    ----------------------------------------''')
    display(data.head())
    print(f'''
    ----------------------------------------
    Последние строки датафрейма:
    ----------------------------------------''')
    display(data.tail())
    print(f'''
    ----------------------------------------
    Общая информация:
    ----------------------------------------''')
    print(data.info())
    print(f'''
    ----------------------------------------
    Дупликаты:
    ----------------------------------------''')
    print(data.duplicated().sum())
    print(f'''
    ----------------------------------------
    Пропуски:
    ----------------------------------------''')
    display(round(data.isna().sum(),))

In [4]:
data_info(df_rough)


    ----------------------------------------
    Первые строки датафрейма:
    ----------------------------------------


,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""



    ----------------------------------------
    Последние строки датафрейма:
    ----------------------------------------


,school_id,name
890,3,"Республика Татарстан, СШОР ФСО Авиатор"
891,3,"СШОР ФСО Авиатор, Республика Татарстан"
892,3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»"
893,2,"ЯНАО, СШ ""Авангард"""
894,1,"Московская область, СШ ""Авангард"""



    ----------------------------------------
    Общая информация:
    ----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  895 non-null    int64 
 1   name       895 non-null    object
dtypes: int64(1), object(1)
memory usage: 14.1+ KB
None

    ----------------------------------------
    Дупликаты:
    ----------------------------------------
0

    ----------------------------------------
    Пропуски:
    ----------------------------------------


school_id    0
name         0
dtype: int64

В таблице имеются 2 колокни: 
- school_id - id школы
- name - собственно название, как оно предположительно будет вбиваться в базу

Датасет небольшой, всего 895 строк, явных дупликатов нет.

In [5]:
data_info(df_clean)


    ----------------------------------------
    Первые строки датафрейма:
    ----------------------------------------


,school_id,name,region
0,1,Авангард,Московская область
1,2,Авангард,Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан
3,4,Аврора,Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург



    ----------------------------------------
    Последние строки датафрейма:
    ----------------------------------------


,school_id,name,region
301,305,Прогресс,Алтайский край
302,609,"""СШ ""Гвоздика""",Удмуртская республика
303,610,"СШОР ""Надежда Губернии",Саратовская область
304,611,КФК «Айсберг»,Пермский край
305,1836,"ООО ""Триумф""",Москва



    ----------------------------------------
    Общая информация:
    ----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  306 non-null    int64 
 1   name       306 non-null    object
 2   region     306 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.3+ KB
None

    ----------------------------------------
    Дупликаты:
    ----------------------------------------
0

    ----------------------------------------
    Пропуски:
    ----------------------------------------


school_id    0
name         0
region       0
dtype: int64

Второй датасет из 306 школ, явных дупликатов нет.

Видим, что обычно, школы пишутся с округом, в котором находятся, поэтому первым делом нужно будет и в *эталонных* названиях их объединить.

Так как школ у нас достаточно немного, необходимо продумать возможность переобучения при обучении на сырых данных. Скорее всего, произойдет чрезмерная подгонка модели под датасет и при новых данных она будет выдавать точность сильно ниже. Поэтому, работать мы будет с помощью аугментации - добавим в "чистый" датасет грязных данных и обучим модель на них. 

In [6]:
df_clean.sort_values('region').iloc[50:100, :]

,school_id,name,region
209,210,СШОР № 2,Курганская область
17,18,Арена,Курская область
34,35,ВСШОР,Ленинградская область
183,184,СШ № 11,Липецкая область
112,113,Новочебоксарск,Москва
74,75,Конек Чайковской,Москва
135,136,ПроСинхро,Москва
105,106,Москвич,Москва
81,82,Легенда,Москва
103,104,МОСГОРСПОРТ,Москва


На данный момент, есть уже несколько неявных дупликатов. Чтобы не сопоставлять названия школ, не гадать, совпадают ли они, неплохо было бы добавить колонку государственной регистрации школ (по типу ИНН или подобного), чтобы не возникало путаницы. На данный момент, имеется датасет без неявных дупликатов, поэтому обучать модели буду на нем:

In [137]:
df_clean = pd.read_csv('C:/Users/Home/Downloads/data_02_match_school/reference_schools.csv', index_col=0).reset_index(drop=True)

In [138]:
data_info(df_clean)


    ----------------------------------------
    Первые строки датафрейма:
    ----------------------------------------


,school_id,name,region
0,1,Авангард,Московская область
1,2,Авангард,Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан
3,4,Аврора,Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург



    ----------------------------------------
    Последние строки датафрейма:
    ----------------------------------------


,school_id,name,region
283,305,Прогресс,Алтайский край
284,609,"""СШ ""Гвоздика""",Удмуртская республика
285,610,"СШОР ""Надежда Губернии",Саратовская область
286,611,КФК «Айсберг»,Пермский край
287,1836,"ООО ""Триумф""",Москва



    ----------------------------------------
    Общая информация:
    ----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  288 non-null    int64 
 1   name       288 non-null    object
 2   region     288 non-null    object
dtypes: int64(1), object(2)
memory usage: 6.9+ KB
None

    ----------------------------------------
    Дупликаты:
    ----------------------------------------
0

    ----------------------------------------
    Пропуски:
    ----------------------------------------


school_id    0
name         0
region       0
dtype: int64

In [139]:
df_clean['name'] = (df_clean.name.replace('No',' ', regex=True)
                    .replace('№',' ',regex=True)
                    .replace(r'^А-Яа-яЁЙ\w\s', ' ', regex=True)
                    .replace('\s+', ' ', regex=True))

#### Этап 2. Аугментация.

In [140]:
df_clean['united'] = df_clean['name'] + ' по ' + df_clean['region']
display(df_clean)
df = df_clean[['school_id','united']]
df

,school_id,name,region,united
0,1,Авангард,Московская область,Авангард по Московская область
1,2,Авангард,Ямало-Ненецкий АО,Авангард по Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан,Авиатор по Республика Татарстан
3,4,Аврора,Санкт-Петербург,Аврора по Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург,Ice Dream / Айс Дрим по Санкт-Петербург
...,...,...,...,...
283,305,Прогресс,Алтайский край,Прогресс по Алтайский край
284,609,"""СШ ""Гвоздика""",Удмуртская республика,"""СШ ""Гвоздика"" по Удмуртская республика"
285,610,"СШОР ""Надежда Губернии",Саратовская область,"СШОР ""Надежда Губернии по Саратовская область"
286,611,КФК «Айсберг»,Пермский край,КФК «Айсберг» по Пермский край


,school_id,united
0,1,Авангард по Московская область
1,2,Авангард по Ямало-Ненецкий АО
2,3,Авиатор по Республика Татарстан
3,4,Аврора по Санкт-Петербург
4,5,Ice Dream / Айс Дрим по Санкт-Петербург
...,...,...
283,305,Прогресс по Алтайский край
284,609,"""СШ ""Гвоздика"" по Удмуртская республика"
285,610,"СШОР ""Надежда Губернии по Саратовская область"
286,611,КФК «Айсберг» по Пермский край


В первую очередь будем тестировать *"нейросеть из коробки"* - SentenceTransformer. Для этого, сделаем аугментацию данных, ибо данных достаточно мало самих по себе. Будем увеличивать и изменять эталонный датасет, по которому и будем обучать модель.

Далее было принято решение сделать 2 самых частых возможных варианта опечаток в тексте:
- когда случайно была введена дополнительная буква (включая оба регистра)
- когда случайно буквы были поменяны местами

In [141]:
aug = nac.RandomCharAug(candidates=['А','Б','В','Г','Д','Е','Ё','Ж','З','И',
                                    'Й','К','Л','М','Н','О','П','Р','С','Т',
                                    'У','Ф','Х','Ц','Ч','Ш','Щ','Ъ','Ы','Ь',
                                    'Э','Ю','Я','а','б','в','г','д','е','ё',
                                    'ж','з','и','й','к','л','м','н','о','п',
                                    'р','с','т','у','ф','х','ц','ч','ш','щ',
                                    'ъ','ы','ь','э','ю','я'],
                       action = 'insert', aug_char_max=2, aug_word_max=2)

df['insert'] = df.united.transform(lambda x: aug.augment(x, n=4))
df = df.explode('insert')
aug = nac.RandomCharAug(candidates=['А','Б','В','Г','Д','Е','Ё','Ж','З','И',
                                    'Й','К','Л','М','Н','О','П','Р','С','Т',
                                    'У','Ф','Х','Ц','Ч','Ш','Щ','Ъ','Ы','Ь',
                                    'Э','Ю','Я','а','б','в','г','д','е','ё',
                                    'ж','з','и','й','к','л','м','н','о','п',
                                    'р','с','т','у','ф','х','ц','ч','ш','щ',
                                    'ъ','ы','ь','э','ю','я'],
                       action = 'swap', swap_mode = 'adjacent', aug_char_max=3)
       
df['aug'] = df['insert'].transform(lambda x: aug.augment(x, n=4))
df = df.explode('aug').reset_index(drop=True).drop('insert',axis=1)
df.head(25)

,school_id,united,aug
0,1,Авангард по Московская область,АовангОард по ЙМоскьовкяса обалсть
1,1,Авангард по Московская область,АонваоГард по ЙМоскьовская боалсьт
2,1,Авангард по Московская область,АовангОард по МЙокьсовская облсьат
3,1,Авангард по Московская область,АоавнгОдар по ЙМоскьовская обалсть
4,1,Авангард по Московская область,Авагандр по ХМоскобвская обласатНь
5,1,Авангард по Московская область,Авангард по МХоскбовскяа олбааснТь
6,1,Авангард по Московская область,Аавгнадр по ХМоскобвская боалсатНь
7,1,Авангард по Московская область,Ваанград по ХМоскобвская олбаастЬн
8,1,Авангард по Московская область,Аавгнадр по чМосковРская иболаЧсть
9,1,Авангард по Московская область,Ваангард по чМосквоСркяа иоблаЧсть


In [142]:
df.shape

(4608, 3)

В итоге, у нас из **300+** строк получилось ***4600***. Получилось по 15 вариантов *различного* написания для одной *эталонной* школы.

In [143]:
dfff = df.copy()

#### Этап 3. Обучение модели.

Пробуем LaBSE:

In [144]:
model = SentenceTransformer('sentence-transformers/LaBSE')

In [145]:
corpus = model.encode(df_clean.united.values)

In [150]:
query = model.encode(df.aug.values)

In [151]:
search_res = semantic_search(query, corpus, top_k=1)

In [156]:
df['candidates_ind'] = [x[0]['corpus_id'] for x in search_res]
df['candidate_names'] = df.candidates_ind.transform(lambda x: df_clean.united.values[x])
df['candidate_id'] = df.candidates_ind.transform(lambda x: df_clean.school_id.values[x])
df.head(10)

,school_id,united,aug,candidates_ind,candidate_names,candidate_id
0,1,Авангард по Московская область,АовангОард по ЙМоскьовкяса обалсть,0,Авангард по Московская область,1
1,1,Авангард по Московская область,АонваоГард по ЙМоскьовская боалсьт,0,Авангард по Московская область,1
2,1,Авангард по Московская область,АовангОард по МЙокьсовская облсьат,0,Авангард по Московская область,1
3,1,Авангард по Московская область,АоавнгОдар по ЙМоскьовская обалсть,0,Авангард по Московская область,1
4,1,Авангард по Московская область,Авагандр по ХМоскобвская обласатНь,0,Авангард по Московская область,1
5,1,Авангард по Московская область,Авангард по МХоскбовскяа олбааснТь,0,Авангард по Московская область,1
6,1,Авангард по Московская область,Аавгнадр по ХМоскобвская боалсатНь,124,Пахомовой по Московская область,129
7,1,Авангард по Московская область,Ваанград по ХМоскобвская олбаастЬн,0,Авангард по Московская область,1
8,1,Авангард по Московская область,Аавгнадр по чМосковРская иболаЧсть,0,Авангард по Московская область,1
9,1,Авангард по Московская область,Ваангард по чМосквоСркяа иоблаЧсть,0,Авангард по Московская область,1


In [154]:
((df.united == df.candidate_names).sum()/df.shape[0])*100

76.5625

76% - результат, достаточно далекий от идеала. Пробуем еще одну модель у sentence-transformers:

In [157]:
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')
corpus = model.encode(df_clean.united.values)
query = model.encode(df.aug.values)

In [158]:
search_res = semantic_search(query, corpus, top_k=1)

In [159]:
df['candidates_ind'] = [x[0]['corpus_id'] for x in search_res]
df['candidate_names'] = df.candidates_ind.transform(lambda x: df_clean.united.values[x])
df['candidate_id'] = df.candidates_ind.transform(lambda x: df_clean.school_id.values[x])
df.head(10)

,school_id,united,aug,candidates_ind,candidate_names,candidate_id
0,1,Авангард по Московская область,АовангОард по ЙМоскьовкяса обалсть,1,Авангард по Ямало-Ненецкий АО,2
1,1,Авангард по Московская область,АонваоГард по ЙМоскьовская боалсьт,1,Авангард по Ямало-Ненецкий АО,2
2,1,Авангард по Московская область,АовангОард по МЙокьсовская облсьат,1,Авангард по Ямало-Ненецкий АО,2
3,1,Авангард по Московская область,АоавнгОдар по ЙМоскьовская обалсть,118,Олимпиец по ХМАО-Югра,123
4,1,Авангард по Московская область,Авагандр по ХМоскобвская обласатНь,118,Олимпиец по ХМАО-Югра,123
5,1,Авангард по Московская область,Авангард по МХоскбовскяа олбааснТь,1,Авангард по Ямало-Ненецкий АО,2
6,1,Авангард по Московская область,Аавгнадр по ХМоскобвская боалсатНь,118,Олимпиец по ХМАО-Югра,123
7,1,Авангард по Московская область,Ваанград по ХМоскобвская олбаастЬн,29,Витязь по Вологодская область,31
8,1,Авангард по Московская область,Аавгнадр по чМосковРская иболаЧсть,92,МАФКК по Москва,96
9,1,Авангард по Московская область,Ваангард по чМосквоСркяа иоблаЧсть,29,Витязь по Вологодская область,31


In [160]:
((df.united == df.candidate_names).sum()/df.shape[0])*100

59.65711805555556

Метрика пошла вниз. Надо посмотреть, как отработает Tf-Idf Vectorizer вместе с опорными векторами или регрессией:

In [161]:
features = df['united']
target = df['school_id']

In [162]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
STOP_WORDS = list(stopwords.words('english'))
# векторизируем наш текст:
vect = TfidfVectorizer(stop_words=list(STOP_WORDS))
tfidf_train = vect.fit_transform(features)
print(tfidf_train.shape)

(4608, 364)


In [163]:
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV

score = make_scorer(accuracy_score, greater_is_better = True)

def return_best_model(model, params, features, target):
    
    pipeline = Pipeline(steps=[
        ('vect', TfidfVectorizer(stop_words=STOP_WORDS)), 
        ('clf', model)])
    
    search = RandomizedSearchCV(pipeline, param_distributions = params, cv=3, 
                                    scoring = score, random_state=12345, verbose=4)
    print('start search')
    search.fit(features, target)
    print(f'best params: {search.best_params_}')
    print(f'best score: {search.best_score_}')
    search.best_estimator_.fit(features, target)
    predictions = search.best_estimator_.predict(features)        
    return search

In [164]:
model = LinearSVC()
params_lsvc = {'clf__C': [0.1, 1, 10],
               'clf__max_iter': [100, 1000, 2000],
               'clf__penalty': ['l1', 'l2'],
               'clf__dual': [True, False]}

search = return_best_model(model, params_lsvc, features, target)

start search
Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3] END clf__C=0.1, clf__dual=True, clf__max_iter=2000, clf__penalty=l2;, score=0.989 total time=   1.2s
[CV 2/3] END clf__C=0.1, clf__dual=True, clf__max_iter=2000, clf__penalty=l2;, score=0.988 total time=   1.2s
[CV 3/3] END clf__C=0.1, clf__dual=True, clf__max_iter=2000, clf__penalty=l2;, score=0.990 total time=   1.2s
[CV 1/3] END clf__C=0.1, clf__dual=True, clf__max_iter=1000, clf__penalty=l1;, score=nan total time=   0.2s
[CV 2/3] END clf__C=0.1, clf__dual=True, clf__max_iter=1000, clf__penalty=l1;, score=nan total time=   0.2s
[CV 3/3] END clf__C=0.1, clf__dual=True, clf__max_iter=1000, clf__penalty=l1;, score=nan total time=   0.2s
[CV 1/3] END clf__C=1, clf__dual=False, clf__max_iter=1000, clf__penalty=l2;, score=0.989 total time=   3.3s
[CV 2/3] END clf__C=1, clf__dual=False, clf__max_iter=1000, clf__penalty=l2;, score=0.988 total time=   2.5s
[CV 3/3] END clf__C=1, clf__dual=False, clf__max_iter=

In [165]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
params_lr = {
    'clf__penalty': ['l1', 'l2'],
    'clf__C': [0.1, 1, 5, 10, 15],
    'clf__solver':['lbfgs', 'liblinear', 'sag', 'saga']}

search = return_best_model(model, params_lsvc, features, target)

start search
Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3] END clf__C=0.1, clf__dual=True, clf__max_iter=2000, clf__penalty=l2;, score=nan total time=   0.0s
[CV 2/3] END clf__C=0.1, clf__dual=True, clf__max_iter=2000, clf__penalty=l2;, score=nan total time=   0.0s
[CV 3/3] END clf__C=0.1, clf__dual=True, clf__max_iter=2000, clf__penalty=l2;, score=nan total time=   0.0s
[CV 1/3] END clf__C=0.1, clf__dual=True, clf__max_iter=1000, clf__penalty=l1;, score=nan total time=   0.0s
[CV 2/3] END clf__C=0.1, clf__dual=True, clf__max_iter=1000, clf__penalty=l1;, score=nan total time=   0.0s
[CV 3/3] END clf__C=0.1, clf__dual=True, clf__max_iter=1000, clf__penalty=l1;, score=nan total time=   0.0s
[CV 1/3] END clf__C=1, clf__dual=False, clf__max_iter=1000, clf__penalty=l2;, score=0.989 total time=   3.6s
[CV 2/3] END clf__C=1, clf__dual=False, clf__max_iter=1000, clf__penalty=l2;, score=0.988 total time=   3.1s
[CV 3/3] END clf__C=1, clf__dual=False, clf__max_iter=1000, 

Будем использовать опорные вектора. Теперь, опробуем модели на сырых данных:

In [176]:
model = SentenceTransformer('sentence-transformers/LaBSE')

In [177]:
corpus = model.encode(df_clean.united.values)

In [178]:
query = model.encode(df_rough.name.values)

In [179]:
search_res = semantic_search(query, corpus, top_k=1)

In [180]:
data = df_rough.copy()

In [182]:
data['candidates_ind'] = [x[0]['corpus_id'] for x in search_res]
data['candidates_names'] = data.candidates_ind.transform(lambda x: df_clean.united.values[x])
data['candidate_id'] = data.candidates_ind.transform(lambda x: df_clean.school_id.values[x])
data.head(25)

,school_id,name,candidates_ind,candidates_names,candidate_id
0,1836,"ООО ""Триумф""",287,"ООО ""Триумф"" по Москва",1836
1,1836,"Москва, СК ""Триумф""",287,"ООО ""Триумф"" по Москва",1836
2,610,"СШОР ""Надежда Губернии",285,"СШОР ""Надежда Губернии по Саратовская область",610
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...",285,"СШОР ""Надежда Губернии по Саратовская область",610
4,609,"""СШ ""Гвоздика""",284,"""СШ ""Гвоздика"" по Удмуртская республика",609
5,304,СШ по ЗВС,166,СШ по ХМАО-Югра,173
6,304,"Пензенская область, СШ по ЗВС",282,СШ по ЗВС по Пензенская область,304
7,304,"Пензенская область, МБУ ДО ""СШ по зимним видам...",282,СШ по ЗВС по Пензенская область,304
8,303,"ООО ""СетПоинт""",281,"ООО ""СетПоинт"" по Москва",303
9,303,"Москва, ООО ""Сетпоинт""",281,"ООО ""СетПоинт"" по Москва",303


In [183]:
((data.school_id == data.candidate_id).sum()/data.shape[0])*100

55.53072625698324

И опорные вектора:

In [184]:
features = df['united']
target = df['school_id']

features_test = df_rough['name']
target_test = df_rough['school_id']

# векторизируем наш текст:
vect = TfidfVectorizer(stop_words=list(STOP_WORDS))
tfidf_train = vect.fit_transform(features)
tfidf_test = vect.transform(features_test)
model = LinearSVC(C = 10, max_iter = 100, penalty = 'l2', dual = True)

In [185]:
model = model.fit(tfidf_train, target)

In [186]:
predictions = model.predict(tfidf_test)

In [251]:
df_rough['cand_school'] = predictions
df_rough['cand_names'] = [df_clean.set_index('school_id').loc[[x]]['united'].values for x in predictions]

In [252]:
df_rough

,school_id,name,cand_school,cand_names
0,1836,"ООО ""Триумф""",1836,"[ООО ""Триумф"" по Москва]"
1,1836,"Москва, СК ""Триумф""",1836,"[ООО ""Триумф"" по Москва]"
2,610,"СШОР ""Надежда Губернии",610,"[СШОР ""Надежда Губернии по Саратовская область]"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...",610,"[СШОР ""Надежда Губернии по Саратовская область]"
4,609,"""СШ ""Гвоздика""",609,"[""СШ ""Гвоздика"" по Удмуртская республика]"
...,...,...,...,...
890,3,"Республика Татарстан, СШОР ФСО Авиатор",3,[Авиатор по Республика Татарстан]
891,3,"СШОР ФСО Авиатор, Республика Татарстан",3,[Авиатор по Республика Татарстан]
892,3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»",3,[Авиатор по Республика Татарстан]
893,2,"ЯНАО, СШ ""Авангард""",240,[ФФКК ЯНАО по Ямало-Ненецкий АО]


In [253]:
accuracy_score(target_test, predictions)*100

79.44134078212291

Модель *'из коробки'* работает неплохо, однако пока что лучше всех справились **опорные вектора** - разница составляет практически в 25%.

#### Этап 4. Результаты, выводы.

В ходе работы нам были даны две таблицы, содержащие эталонные названия школ и примерное их написание по факту. Стояла задача - подобрать модель и алгоритм, который будет сопоставлять написанные названия школ с эталонными. Обучив нейросеть SentenceTransformers на чистых обработанных данных, получили точность в 76%, что достаточно далеко от идеала. Обучив метод опорных векторов и логистическую регрессию, мы получили результат в разы лучше - практически 99% совпадений.